In [109]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests
import fuzzywuzzy
from fuzzywuzzy import fuzz,process



In [4]:
#conection_string='mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
#engine = create_engine(conection_string)

In [5]:
#inspector = inspect(engine)
#inspector.get_table_names()

In [110]:
#df_stations = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
df_stations=pd.read_csv('./bicimad_stations_202207091252.csv')

In [111]:
#coordinates=df_stations["geometry.coordinates"][0].strip('][').split(',')
df_stations[["longitude","latitude"]]=df_stations["geometry.coordinates"].str.split(",",expand=True)
df_stations['longitude'] = df_stations['longitude'].str.replace('[','',regex=True)
df_stations["latitude"]=df_stations['latitude'].str.replace(']','',regex=True)
df_stations['latitude'] = pd.to_numeric(df_stations['latitude'])
df_stations['longitude'] = pd.to_numeric(df_stations['longitude'])

In [112]:
url='/catalogo/208844-0-monumentos-edificios.json'
end_point='https://datos.madrid.es/egob'
response = requests.get(end_point+url)
mon_json = response.json()

In [113]:
mon_json['@graph']
#la key donde esta toda la info es @graph
df_monuments = pd.DataFrame(mon_json['@graph'])

In [114]:
df_monuments = pd.json_normalize(mon_json['@graph']).dropna(subset=['location.latitude','location.longitude'])

In [14]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c



In [15]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

## ONE MONUMENT

In [116]:
monument=input('Monument name: ')
match_monument = process.extractOne(monument,df_monuments['title'], score_cutoff=80)
z = df_monuments.loc[df_monuments['title']== match_monument[0]]
n=z.iloc[0]['title']
select_monument = df_monuments.loc[df_monuments['title']== n].reset_index(drop=True)

Monument name: banco


In [117]:
df_stations['distance']=df_stations.apply(lambda x: distance_meters(x['latitude'], 
                                                                    x['longitude'],
                                                                    select_monument[('location.latitude')],
                                                                    select_monument[('location.longitude')]),axis=1)
df_stations_1=df_stations[df_stations['distance']==df_stations['distance'].min()].reset_index(drop=True)

In [118]:
df_output=select_monument[['title','address.street-address']].join(df_stations_1[['name','address']])
df_output

,title,address.street-address,name,address
0,Banco Bilbao Vizcaya Argentaria,Calle ALCALÁ 16,28 - Sevilla,Calle Alcalá nº 27


## ALL MONUMENTS

In [221]:
#ALL_monuments=df_monuments[:6][['title','address.street-address','location.latitude','location.longitude']].reset_index(drop=True)
ALL_monuments = df_monuments[['title','address.street-address','location.latitude','location.longitude']].reset_index(drop=True)

In [222]:
ALL_MONUMENTS=[]
for index, row in ALL_monuments.iterrows():
    df_stations['distance']=df_stations.apply(lambda x: distance_meters(x['latitude'], 
                                                                    x['longitude'],
                                                                    row[('location.latitude')],
                                                                    row[('location.longitude')]),axis=1)
    df1=df_stations[df_stations['distance']==df_stations['distance'].min()].reset_index(drop=True)
    df1['title']=row['title']
    #display(df1)
    ALL_MONUMENTS.append(df1)
df_final = pd.concat(ALL_MONUMENTS)

In [223]:
df_output2= pd.merge(ALL_monuments[['title','address.street-address']], df_final[['name','address','title']], on ='title')
df_output2

,title,address.street-address,name,address
0,Almacenes Rodríguez,Calle CABALLERO DE GRACIA 3,18 - Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
1,Antigua Fábrica Martini &amp; Rossi,AVENIDA ARAGON 328,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77
2,Ateneo de Madrid,Calle PRADO 21,52 - Plaza de Santa Ana,Plaza de Santa Ana nº 10
3,Banco Bilbao Vizcaya Argentaria,Calle ALCALÁ 16,28 - Sevilla,Calle Alcalá nº 27
4,Banco de España,Calle ALCALÁ 48,20a - Banco de España A,Calle Alcalá nº 49
...,...,...,...,...
223,Viviendas y oficinas para Ramón Saiz de Carlos,Calle GRAN VIA 33,2 - Miguel Moya,Calle Miguel Moya nº 1
224,Viviendas y oficinas para Santos Suárez y Comp...,Calle GRAN VIA 47,23 - Santo Domingo,Plaza de Santo Domingo nº 1
225,Viviendas y oficinas para Seguros La Estrella,Calle CABALLERO DE GRACIA 15,28 - Sevilla,Calle Alcalá nº 27
226,Viviendas y teatro Fontalba para el marqués de...,Calle DESENGAÑO 3,211 - Desengaño,Calle Desengaño nº 1


In [108]:
#EXTRA

import folium

df_map = df_monuments
m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)
m
##The m will return a map of the world that is zoomable.

##To add the markers to this:

for i in range(0,len(df_map)):
    folium.Marker(
      location=[df_map.iloc[i]['location.latitude'], df_map.iloc[i]['location.longitude']],
      popup=df_map.iloc[i]['title']).add_to(m)
m